# STEP 0: Use GPU

In [1]:
# Make sure you use the Colab GPU to run the testing phase

import torch
torch.cuda.is_available()

True

In [2]:
# Show the GPU info

!nvidia-smi

Thu Nov 25 17:23:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# STEP 1: Git clone your project

In [3]:
# Git clone your project

!git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 21799, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 21799 (delta 5), reused 9 (delta 3), pack-reused 21781
Receiving objects: 100% (21799/21799), 25.21 MiB | 22.88 MiB/s, done.
Resolving deltas: 100% (15308/15308), done.


# STEP 2: Install your requirements

In [4]:
# Install your requirments (torch, mmcv, ...)
# It is recommended that you wirte the requirements.txt file in your project.

import os
import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle

os.chdir('/content/mmdetection')

!pip install -r requirements/build.txt
!pip install "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"
!pip install -v -e .  # or "python setup.py develop"

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html



  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-ki2l32lj
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-ki2l32lj
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263926 sha256=b6ab350da219e288f38ea600165495507f57f3f66d090202c1f2683c769e68c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-6rnqp73w/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not matc

Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
     |████████████████████████████████| 58.0 MB 3.8 MB/s 
     |████████████████████████████████| 185 kB 7.9 MB/s 


In [6]:
# Import your package and check the version

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmdet
print(mmdet.__version__)

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# You must import the below 5 packages 
import os
import cv2
import time
from tqdm import tqdm
from google_drive_downloader import GoogleDriveDownloader as gdd

1.10.0+cu111 True
2.18.1
11.1
GCC 7.3


# STEP 3: Wget testing data 

In [7]:
# 1. Download the testing data

gdd.download_file_from_google_drive(file_id='1Fm-avdeNgzhPxhvia0iw9yZzcoOggy7I',
                                    dest_path='./test.zip',
                                    unzip=True)

# 2. Read the image_name and put them into the list
# You need to modify the path to fit your test_folder
data_listdir = os.listdir("/content/mmdetection/test")


Unzipping...Done.


# STEP 4: Run inferene and bench mark

In [9]:
os.chdir('/content/mmdetection/configs/faster_rcnn')
!gdown --id '1qJwljvoxLXE1r8GAD_oMnxQQWhXo9OU_' --output VRDL_HW2_model.zip
!apt-get install unzi
!unzip 'VRDL_HW2_model.zip'

!gdown --id '13CGHs-8s5WV-voRt1FDhjmlCsn_7WmZA'

Downloading...
From: https://drive.google.com/uc?id=1qJwljvoxLXE1r8GAD_oMnxQQWhXo9OU_
To: /content/mmdetection/configs/faster_rcnn/VRDL_HW2_model.zip
100% 307M/307M [00:02<00:00, 103MB/s]
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package unzi
Archive:  VRDL_HW2_model.zip
  inflating: VRDL_HW2.pth            
Downloading...
From: https://drive.google.com/uc?id=13CGHs-8s5WV-voRt1FDhjmlCsn_7WmZA
To: /content/mmdetection/configs/faster_rcnn/VRDL_HW2_train_config.py
100% 1.88k/1.88k [00:00<00:00, 3.22MB/s]


In [12]:
# 1. Load your model and weights
os.chdir('/content/mmdetection')
from mmcv import Config
cfg = Config.fromfile('/content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
os.chdir('/content/mmdetection')
from mmdet.apis import set_random_seed

classes = ('0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0')
# Modify dataset type and path
cfg.dataset_type = 'HW2_SVHN_Dataset'
cfg.data_root = '2021VRDL_HW2/'

cfg.data.test.type = 'HW2_SVHN_Dataset'
cfg.data.test.data_root = '2021VRDL_HW2/'
cfg.data.test.ann_file = '2021VRDL_HW2_train/train/digitStruct.mat'
cfg.data.test.img_prefix = '2021VRDL_HW2_test/test/'
cfg.data.test.classes = classes

cfg.data.train.type = 'HW2_SVHN_Dataset'
cfg.data.train.data_root = '2021VRDL_HW2/'
cfg.data.train.ann_file = '2021VRDL_HW2_train/train/digitStruct.mat'
cfg.data.train.img_prefix = '2021VRDL_HW2_train/train/'
cfg.data.train.classes = classes

cfg.data.val.type = 'HW2_SVHN_Dataset'
cfg.data.val.data_root = '2021VRDL_HW2/'
cfg.data.val.ann_file = '2021VRDL_HW2_train/train/digitStruct.mat'
cfg.data.val.img_prefix = '2021VRDL_HW2_train/train/'
cfg.data.val.classes = classes

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 11
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_bbox_mAP-0.398_20200504_163323-30042637.pth'  # noqa


# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 3
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

checkpoint = '/content/mmdetection/configs/faster_rcnn/VRDL_HW2.pth'

# 2. Initialize the model
os.chdir('/content/mmdetection')
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
model = init_detector(cfg, checkpoint, device='cuda:0')


load checkpoint from local path: /content/mmdetection/configs/faster_rcnn/VRDL_HW2.pth


/content/mmdetection/mmdet/apis/inference.py:47: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [14]:
# Test your inference time
TEST_IMAGE_NUMBER = 100 # This number is fixed.
test_img_list = []

# Read image (Be careful with the image order)
data_listdir.sort(key = lambda x: int(x[:-4]))
# [:TEST_IMAGE_NUMBER]
for img_name in data_listdir[:TEST_IMAGE_NUMBER]:
  img_path = os.path.join("/content/mmdetection/test", img_name)
  img = cv2.imread(img_path)
  test_img_list.append(img)

start_time = time.time()
for img in tqdm(test_img_list):
    # your model prediction
    pred = inference_detector(model, img)

end_time  = time.time()
print("\nInference time per image: ", (end_time - start_time) / len(test_img_list))

# Remember to screenshot!

100%|██████████| 100/100 [00:08<00:00, 12.31it/s]


Inference time per image:  0.08129149436950683
